In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.svm import SVC
from scipy.io import arff

In [ ]:
data = arff.loadarff("data/rice.arff")
df = pd.DataFrame(data[0])
df.head()

In [ ]:
df.info()

In [ ]:
df['Class'].unique()

In [ ]:
df['Class'] = df['Class'].map({b'Cammeo':0, b'Osmancik':1})
df['Class'].unique()

In [ ]:
df.isnull().sum()

In [ ]:
X = df.drop(columns=['Class'])
y = df['Class']

In [ ]:
X = PCA(n_components=2).fit_transform(X)

In [ ]:
X.shape

In [ ]:
plt.scatter(X[:,0], X[:,1], c=y, edgecolor="black")
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
models=[
    SVC(kernel='linear', C=1, degree=5, gamma="auto", probability=True),
    SVC(kernel='rbf', C=1, gamma='scale', probability=True),
    SVC(kernel='poly', C=1, degree=5, gamma="auto", probability=True)
]

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc
from mlxtend.plotting import plot_decision_regions

for model in models:
    nonlinear_svm = make_pipeline(StandardScaler(), model)
    nonlinear_svm.fit(X_train, y_train)
    y_pred = nonlinear_svm.predict(X_test)
    print("SVM Classification Report:")
    print(classification_report(y_test, y_pred))
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    plot_decision_regions(X=X_train, 
                          y=y_train.values,
                          clf=nonlinear_svm, 
                          legend=2)

    plt.title('SVM Decision Region Boundary', size=16)
    plt.show()
    conf=confusion_matrix(y_test,y_pred)
    ConfusionMatrixDisplay(conf,display_labels=[0,1]).plot(cmap='Blues')
    plt.show()
    y_pred_proba=nonlinear_svm.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba) 
    roc_auc = auc(fpr, tpr)
    
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--', label='No Skill')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve for Rice Classification')
    plt.legend()
    plt.show()